In [20]:
import numpy as np
import pandas as pd
from array import array
import uproot
import csv
from glob import glob
from pathlib import Path
import ROOT



In [6]:
# .txt -> .csv 

absolute_path_only = '/media/droc/Seagate/PHYS-461/4-hall_effect/data/'
input_files = glob(absolute_path_only + '/*.txt')
# print(input_files)

base_csv = []
for file in input_files:
    base = Path(file).stem
    base_csv.append(base + '.csv')

input_path = []
output_path = []

for i in range(len(base_csv)):
    input_file_path = input_files[i]
    output_file_path = absolute_path_only + '/' + base_csv[i]

    input_path.append(input_file_path)
    output_path.append(output_file_path)
    
    # print(output_file_path)

    with open(input_file_path, 'r') as txt_file, open(output_file_path, 'w', newline='') as csv_file:
        reader = txt_file.readlines()
        writer = csv.writer(csv_file)
        # print(writer)

        for line in reader:
            
            
            row = line.split()
            writer.writerow(row)
            

In [124]:
# .csv -> .root

out_root = []
for file in output_path:

    df = pd.read_csv(file)

    file_out_root = file.replace('.csv','.root')
    out_root.append(file_out_root)
    root_file = ROOT.TFile(file_out_root, "RECREATE")
    tree = ROOT.TTree("data", "Data from CSV")

    branches = {}

    def clean_data(df):
        for col in df.columns:
            # Convert all values in the column to float, set non-convertible values to NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')
        return df

    df = clean_data(df)

    for column in df.columns:
        branches[column] = array('f', [0])  # Create float arrays for each column
        tree.Branch(column, branches[column], f'{column}/F')

    for index, row in df.iterrows():
        for column in df.columns:
            branches[column][0] = row[column]
        tree.Fill()

    # Write the tree to the ROOT file and close it
    tree.Write()
    root_file.Close()

    # print(root_file)

In [127]:
file = ROOT.TFile(out_root[-1])
print(file)

# Access a specific tree
tree = file.Get("data")

# Loop over the entries in the tree and print the values of a branch
for entry in tree:
    # print(entry)
    print(entry.Bnom)  # Replace 'branch_name' with the actual branch name

# Close the file
file.Close()

Name: /media/droc/Seagate/PHYS-461/4-hall_effect/data//Si_nDoped_.5mA_run4.root Title: 
1.9907000064849854
2.9572999477386475
3.785799980163574
4.6819000244140625
5.579400062561035
6.476900100708008
7.374499797821045
8.27340030670166
9.13759994506836
10.04800033569336
nan
